In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [ ]:
# Load the AIS Data
data = pd.read_csv(r"/content/FeatureEngineering - Sheet1.csv")

# Inspect your data
print(data.head())
print(data.info())
print(data.describe())
print(data.isna().sum())

        MMSI        DepartureTime      LATd      LONd          ArrivalTime  \
0  205366000  2017-11-26 19:51:00  57.40660  21.53463  2017-11-26 19:51:00   
1  205366000  2017-11-27 13:18:00  57.40660  21.53437  2017-12-28 13:57:00   
2  205366000   2018-06-02 7:38:00  54.16003  12.14130   2018-06-03 5:43:00   
3  205366000  2018-06-03 12:16:00  57.59602  11.69184   2018-06-08 5:43:00   
4  205366000   2018-09-15 8:27:00  59.42259  27.72447  2018-10-01 15:16:00   

       LATa      LONa  AVGSPDkmph  DistanceKm  VesselType  Length  Width  \
0  57.40660  21.53463       28.65   723.43028        80.0   143.0   21.0   
1  54.16002  12.14124        2.73  2030.43443        80.0   143.0   21.0   
2  57.59724  11.69437       21.09   465.67234        80.0   143.0   21.0   
3  54.16005  12.14124        5.49   622.78374        80.0   143.0   21.0   
4  54.16008  12.14128        6.03  2356.37863        80.0   143.0   21.0   

   Draft  
0    8.2  
1    8.2  
2    8.2  
3    8.2  
4    8.2  
<class '

In [ ]:
# Convert date-time columns
data['DepartureTime'] = pd.to_datetime(data['DepartureTime'], format="%Y-%m-%d %H:%M:%S")
data['ArrivalTime'] = pd.to_datetime(data['ArrivalTime'], format="%Y-%m-%d %H:%M:%S")

# Handle missing values
data.dropna(inplace=True)

# Convert 'ArrivalTime' to Unix timestamps for the SVM
data['ArrivalTime'] = data['ArrivalTime'].astype(np.int64) // 10**9

In [ ]:
# Select relevant features
X = data[['LATd', 'LONd', 'LATa', 'LONa', 'AVGSPDkmph', 'DistanceKm', 'DepartureTime', 'VesselType', 'Length', 'Width', 'Draft']]
y =data[['ArrivalTime']]

# Convert 'DepartureTime' to Unix timestamps for standardization
X['DepartureTime'] = pd.to_datetime(X['DepartureTime']).astype(int) // 10**9

# Standardize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

<ipython-input-4-cc5728de2096>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['DepartureTime'] = pd.to_datetime(X['DepartureTime']).astype(int) // 10**9


In [ ]:
# Define parameter grid for GridSearchCV
param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf'], 'gamma': [0.01, 0.1, 1]}

# Create SVR model
svr = SVR()

# Perform grid search to find best parameters
grid_search = GridSearchCV(svr, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exam

GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': [0.1, 1, 10], 'gamma': [0.01, 0.1, 1],
                         'kernel': ['linear', 'rbf']},
             scoring='neg_mean_squared_error')

In [ ]:
# Get the best model from grid search
best_svr = grid_search.best_estimator_

# Train the best model on full training data
best_svr.fit(X_train, y_train)

# Make predictions
y_pred = best_svr.predict(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)
print("R-squared:", r2)

Mean Absolute Error: 9383717.136979543
Mean Squared Error: 127961832824061.11
R-squared: -0.06801250119603153
